<a href="https://colab.research.google.com/github/ColmTalbot/GPUCBC/blob/master/examples/gpucbc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU-enabled parameter estimation for compact binary coalescences

In this notebook I demonstrate how to use the `GPUCBC` package to perform parameter estimation on very long duration binary neutron star inspirals.

See the [paper](https://arxiv.org/abs/1904.02863) and [repository](https://github.com/ColmTalbot/GPUCBC) for more details.

In this example we will use the implementation of the `TaylorF2` waveform which implements the post-Newtonian expansion for the inspiral of a binary neutron star system including aligned spins and the effect of tidal deformation.

Even with the GPU acceleration the contents of this notebook will take up to a day to run.

# Install requirements

For this example we'll need to install `GPUCBC` from git and `cupy` which is not listed as a requirement.

This will install the dependencies of `GPUCBC` including:
- `lalsuite` for the post-Newtonian coefficients for our waveform.
- `bilby` for interfacing with samplers for Bayesian inference.
- `dynesty` the nested sampler we'll use in this example.

In [1]:
!pip install --upgrade git+https://github.com/ColmTalbot/GPUCBC.git
!pip install cupy --no-cache-dir

  Cloning https://github.com/ColmTalbot/GPUCBC.git to /tmp/pip-req-build-tu9cyxxo
  Running command git clone -q https://github.com/ColmTalbot/GPUCBC.git /tmp/pip-req-build-tu9cyxxo
     |████████████████████████████████| 1.5MB 4.0MB/s 
     |████████████████████████████████| 30.3MB 41.6MB/s 
     |████████████████████████████████| 92kB 29.3MB/s 
     |████████████████████████████████| 1.6MB 37.0MB/s 
     |████████████████████████████████| 51kB 20.2MB/s 
     |████████████████████████████████| 61kB 30.4MB/s 
     |████████████████████████████████| 2.3MB 34.9MB/s 
     |████████████████████████████████| 102kB 27.9MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-g6i8vg7c/wheels/95/14/80/0a84970a2145273710adbfc549096aaa71c4502186256178fe
  Stored in directory: /root/.cache/pip/wheels/8e/52/62/b1f40616bad4daa942230b41fcf53800e9460a82ea3c4e43d9
  Stored in directory: /root/.cache/pip/wheels/70/d8/e5/e0e7974a2a5757483ea5a180c937041cf6872dc9993d78234a
  Stored in directory: /root/.cac

# Imports

In [2]:
%pylab inline

import cupy as xp

import bilby

from gpucbc.likelihood import CUPYGravitationalWaveTransient
from gpucbc.waveforms import TF2WFG

Populating the interactive namespace from numpy and matplotlib


05:24 bilby INFO    : Running bilby version: 0.5.3:
05:24 bilby WARNING : You do not have gwpy installed currently. You will  not be able to use some of the prebuilt functions.
05:24 bilby WARNING : You do not have gwpy installed currently. You will  not be able to use some of the prebuilt functions.
05:24 bilby WARNING : You do not have gwpy installed currently. You will  not be able to use some of the prebuilt functions.


# Define the system we are going to analyse

For demonstration purposes we will simulate a binary neutron inspiral.

This is the usual `bilby` operation of creating a `WaveformGenerator` and `Interferometer` objects.

Here our `WaveformGenerator` comes from `gpucbc`.
Since the injection requires numpy arrays to be used we explicitly cast the signal to be injected to a numpy array.

We inject into zero noise in order to demonstrate that the analysis is unbiased in this case where we know the maximum likelihood point should be the true value.

In [8]:
injection_parameters = dict(
    chirp_mass=1.3, mass_ratio=0.9, chi_1=0.02, chi_2=0.03,
    redshift=0.025, theta_jn=0.4, psi=2.659, phase=0, geocent_time=126,
    ra=1.375, dec=-1.2108, lambda_1=500, lambda_2=600)

duration = 512
sampling_frequency = 2048
minimum_frequency = 15

waveform_arguments = dict(
    reference_frequency=minimum_frequency, minimum_frequency=minimum_frequency)

wfg = TF2WFG(
    duration=duration, sampling_frequency=sampling_frequency,
    waveform_arguments=waveform_arguments)

injection_pols = wfg.frequency_domain_strain(injection_parameters)

for key in injection_pols:
    injection_pols[key] = xp.asnumpy(injection_pols[key])

ifos = bilby.gw.detector.InterferometerList(["H1", "L1", "V1"])
for ifo in ifos:
    ifo.minimum_frequency = minimum_frequency
ifos.set_strain_data_from_zero_noise(
    duration=duration, sampling_frequency=sampling_frequency)
_ = ifos.inject_signal(
    injection_polarizations=injection_pols, parameters=injection_parameters)

05:26 bilby INFO    : Injected signal in H1:
05:26 bilby INFO    :   optimal SNR = 22.46
05:26 bilby INFO    :   matched filter SNR = 22.46+0.00j
05:26 bilby INFO    :   chirp_mass = 1.3
05:26 bilby INFO    :   mass_ratio = 0.9
05:26 bilby INFO    :   chi_1 = 0.02
05:26 bilby INFO    :   chi_2 = 0.03
05:26 bilby INFO    :   redshift = 0.025
05:26 bilby INFO    :   theta_jn = 0.4
05:26 bilby INFO    :   psi = 2.659
05:26 bilby INFO    :   phase = 0
05:26 bilby INFO    :   geocent_time = 126
05:26 bilby INFO    :   ra = 1.375
05:26 bilby INFO    :   dec = -1.2108
05:26 bilby INFO    :   lambda_1 = 500
05:26 bilby INFO    :   lambda_2 = 600
05:26 bilby INFO    : Injected signal in L1:
05:26 bilby INFO    :   optimal SNR = 17.04
05:26 bilby INFO    :   matched filter SNR = 17.04+0.00j
05:26 bilby INFO    :   chirp_mass = 1.3
05:26 bilby INFO    :   mass_ratio = 0.9
05:26 bilby INFO    :   chi_1 = 0.02
05:26 bilby INFO    :   chi_2 = 0.03
05:26 bilby INFO    :   redshift = 0.025
05:26 bilby

# Prior

Define the prior distribution we'll use.
Here we'll start with the default `bilby` prior for analyzing  binary neutron star signals.

We make a few changes to increase the efficiency of the sampling:
- assume the sky location of the signal is known.
This is reasonable for offline analysis of binary neutron star events when a host galaxy has been identified.
- set a narrow gaussian prior on redshift, again assuming that the host galaxy has been identified.
- specify a prior in chirp mass, $\mathcal{M}$, and mass ratio rather than component masses.
This is a much simpler space to sample than the narrow banana shaped posterior in the component mass posterior.
- similarly, we sample in the two leading order tidal terms $\tilde{\Lambda}$ and $\delta \tilde{\Lambda}$ for the same reason.
- we then set prior constraints on the component masses and tidal deformabilities.

None of these are essential to sample the space, however not using them will increase the run time to sample well.


In [12]:
priors = bilby.gw.prior.BNSPriorDict()
priors['ra'] = injection_parameters['ra']
priors['dec'] = injection_parameters['dec']
priors['redshift'] = bilby.core.prior.Gaussian(
    mu=injection_parameters['redshift'],
    sigma=injection_parameters['redshift'] / 10)
del priors['luminosity_distance']

priors['geocent_time'] = bilby.core.prior.Uniform(
    minimum=125.99, maximum=126.01, latex_label='$t_{c}$',
    boundary='reflective')

priors['chirp_mass'] = bilby.core.prior.Uniform(
    minimum=1.299, maximum=1.301, latex_label='$\\mathcal{M}$',
    boundary='reflective')
priors['mass_ratio'] = bilby.core.prior.Uniform(
    minimum=0.5, maximum=1, latex_label='$q$', boundary='reflective')

priors['lambda_tilde'] = bilby.core.prior.Uniform(
    minimum=0, maximum=5000, latex_label='$\\tilde{\\Lambda}$',
    boundary='reflective')
priors['delta_lambda_tilde'] = bilby.core.prior.Uniform(
    minimum=-5000, maximum=5000, latex_label='$\\delta\\tilde{\\Lambda}$',
    boundary='reflective')

priors['mass_1'] = bilby.core.prior.Constraint(minimum=1, maximum=3)
priors['mass_2'] = bilby.core.prior.Constraint(minimum=1, maximum=3)
priors['lambda_1'] = bilby.core.prior.Constraint(minimum=0, maximum=5000)
priors['lambda_2'] = bilby.core.prior.Constraint(minimum=0, maximum=5000)

05:27 bilby INFO    : No prior given, using default BNS priors in /usr/local/lib/python3.6/dist-packages/bilby/gw/prior_files/binary_neutron_stars.prior.
/usr/local/lib/python3.6/dist-packages/bilby/gw/prior.py:234: RuntimeWarning: divide by zero encountered in true_divide
  z_prior.prob(x / aas)), aas) for x in xx]
/usr/local/lib/python3.6/dist-packages/bilby/gw/prior.py:234: RuntimeWarning: invalid value encountered in multiply
  z_prior.prob(x / aas)), aas) for x in xx]


# Define GPU likelihood

Creating the likelihood works in the same way as defining the `bilby` `GravitationalWaveTransient` likelihood.

Distance marginalisation in the likelihood has been implemented, we will not use it here.

In [13]:
like = CUPYGravitationalWaveTransient(
    interferometers=ifos, waveform_generator=wfg, priors=priors,
    distance_marginalization=False)

like.parameters.update(injection_parameters)
print(f"The likelihood at the injected value is {}like.log_likelihood_ratio()")

486.39425329303197


# Run the sampler

Here we run the sampler, these settings have been tuned so the run will take a few hours.

In [0]:
result = bilby.run_sampler(
    likelihood=like, priors=priors, sampler='dynesty', nlive=500,
    walks=50, label=f"cupy_tides_{minimum_frequency}", use_ratio=True,
    injection_parameters=injection_parameters)

# result.injection_parameters = bilby.gw.conversion.generate_all_bns_parameters(
#     injection_parameters)

result.plot_corner()


05:27 bilby INFO    : Running for label 'cupy_tides_15', output will be saved to 'outdir'
05:27 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
05:27 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
05:27 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
05:27 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
05:27 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
05:27 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
05:27 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
05:27 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
05:27 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
05:27 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy


 4238| logz ratio=325.126 +/-  0.207 | dlogz: 96.033 >  0.100